In [8]:
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Prince book exercises chapter 12 - NLP
## 12.1
```markdown
Problem 12.1 Consider a self-attention mechanism that processes N inputs of length D to
produce N outputs of the same size. How many weights and biases are used to compute the
queries, keys, and values? How many attention weights a[•, •] will there be? How many weights
and biases would there be in a fully connected shallow network relating all DN inputs to all DN
outputs?
```

We have 
* input $D \times N$
* output $D \times N$
* query Q, value V, key K all are $D \times D$

Each of Q, V, K uses one bias matrix of size $D$, that is broadcast to $D \times D$, but only D is learnable, the rest are copied. So in total 3 bias matrices, making it $3D$ parameters. Prince defined each bias initially as a column vector that is broadcast to meet the $D \times D$ requirement.

As for the weights of Q,V,K the matrices are all of size $D \times D$, in total $3 \times D \times D$.

Attention weight is given by $\text{Softmax}[K^TQ]$, where $K^T$ is $N \times D$, so we get $N \times N$. **Note that these are NOT learnable**, they are simply the result of taking the softmax of the matrix multiplication between K and Q.

The last sentence is a bit vague, but it sounds like the author wants us to compare the weights of a fully connected network of one layer? If this is the case then the total number of weights including the biases is given by $(DN)^2 + DN = D^2N^2 + DN$, since each neuron in the fully connected layer must be connected to the entire dimension of the input, and there are DN neurons because we want DN outputs (each neuron can only give one output). This grows $\mathcal{O}(N^2)$ with the sequence length N. Compare this to the self-attention that requires in total $3D^2 + 3D$ weights, which is clearly independent from the sequence length as far as learnable weights go (attention matrix is $N \times N$, so grows $\mathcal{O}(N^2)$ with seq length N, but doesn't contain any learnable weights), so it only scales quadratically $\mathcal{O}(D^2)$ w.r.t D and not N.


## 12.2
```markdown
Why might we want to ensure that the input to the self-attention mechanism is the same size as the output? 
```

I think the simple answer is to make the attention weights meaningful to the value matrix, since the attention matrix is derived from Q and K that in turn creates relative similarities between the inputs.


## 12.3
```markdown
Show that the self-attention mechanism (equation 12.8) is equivariant to a permutation XP of the data X, where P is a permutation matrix. In other words, show that: 

                        Sa[XP] = Sa[X]P
```

Since P is a permutation matrix it follows that
$$
B_{*} + \Omega_{*}XP = (B_{*} + \Omega_{*}X)P
$$

where * is taken to mean any of K,Q,V. Now decomposing equation 12.8 into it's components we get the following where I omitted softmax until the very last expression for brevity

$$
\begin{align}
\text{Sa}[XP] &= 
(B_v + \Omega_v XP) (B_k + \Omega_k XP)^T (B_Q + \Omega_Q XP) \\
&= (B_v + \Omega_v X)P [(B_k + \Omega_k X)P]^T (B_Q + \Omega_Q X)P \\
&= (B_v + \Omega_v X)P [P^T (B_k + \Omega_k X)^T] (B_Q + \Omega_Q X)P \\
&= (B_v + \Omega_v X)PP^T (B_k + \Omega_k X)^T (B_Q + \Omega_Q X)P \\
&= (B_v + \Omega_v X) (B_k + \Omega_k X)^T (B_Q + \Omega_Q X)P \\
&= V \cdot \text{Softmax}(K^T Q)P \\
&= \text{Sa}[X]P
\end{align}
$$

This concludes the proof.

The three most important insights in order to prove this are
1. The first relation made at the start is intuitive, where permuting the X first or after will give the same result. You can think of the permutation matrix as permuting the columns and as a consequence the rows as well. If we focus on the permutation of columns, then it's easy to see with simple examples that permuting columns and then doing matrix multipliation versus doing the matrix multiplication and then permuting the columns is the same thing.
2. Permutation matrix P is orthogonal meaning $P^{-1} = P^T$.
3. $[(B_k + \Omega_k X)P]^T = [P^T(B_k + \Omega_k X)^T]$. This makes it possible to cancel the P and P^T, since P is orthogonal matrix.

The implication of this is that tasks that utilizes self-attention should be careful if position of the data matters, such as in NLP (The woman killed the racoon is very different than The racoon killed the woman). The way to solve this is to use position embeddings.

## 12.4
Consider the softmax operation: 

$$
y_i = \text{softmax}_i[z] = \dfrac{\exp{z_i}}{\sum_{j=1}^{5} \exp{z_j}}
$$

in the case where there are five inputs with values: $z_1 = −3, z_2 = 1, z_3 = 100, z_4 = 5, z_5 = −1$. Compute the 25 derivatives, $\dfrac{\partial y_i}{\partial z_j}$ for all $i, j \in \{1, 2, 3, 4, 5\}$. What do you conclude? 

In [10]:
z = np.array([-3, 1, 100, 5, -1])
z.shape

def softmax(z):
    # this is for overflow issues
    # one can show that you can factor out the max term 
    # and they will cancel each other
    # so it's the same as the original softmax
    # however, I think the numerator can still suffer from underflows?
    return np.exp((z - max(z))) / np.exp(z - max(z)).sum(axis=0)
    #return np.exp(z) / np.exp(z).sum(axis=0)
    
# derivative of softmax can be expressed in matrix form as 
# diag(y) - yy^T, where y is the output from softmax
y = softmax(z)
y
np.diag(y) - np.outer(y,y)

(5,)

array([1.85211677e-45, 1.01122149e-43, 1.00000000e+00, 5.52108228e-42,
       1.36853947e-44])

array([[ 1.85211677e-45, -1.87290028e-88, -1.85211677e-45,
        -1.02256891e-86, -2.53469490e-89],
       [-1.87290028e-88,  1.01122149e-43, -1.01122149e-43,
        -5.58303706e-85, -1.38389653e-87],
       [-1.85211677e-45, -1.01122149e-43,  0.00000000e+00,
        -5.52108228e-42, -1.36853947e-44],
       [-1.02256891e-86, -5.58303706e-85, -5.52108228e-42,
         5.52108228e-42, -7.55581902e-86],
       [-2.53469490e-89, -1.38389653e-87, -1.36853947e-44,
        -7.55581902e-86,  1.36853947e-44]])

We can clearly see that an outlier like 100 as entry relative to it's other entries in the $z$ vector makes the weights unbalanced, so that almost all probabilities are assigned to the outlier. In the derivative this is reflected as zero in value while the other derivatives have very low values to the magnitude of $10^{-45}$ and $10^{-85}$. We can conclude that data needs to the normalized before going through a softmax, otherwise the probabilities will be heavily skewed by the outliers. This also explains why we need to perform scaled dot products, because the normal way is to normalize the dot products since they are susceptible to be affected by the magnitude of the vectors.

Proof for the derivation of softmax can be found here

![](assets/12-4-exercise-proof.png)

# 12.5
Why is implementation more efficient if the values, queries, and keys in each of the H heads each have dimension D/H where D is the original dimension of the data?

As already established in [12.1](#12.1) the weights needed are $\mathcal{O}(D^2)$, independent on the sequence length N. For multiple heads h we would get the same but with factor h now $\mathcal{O}(D^2H)$ if the dimensions D remained the same (easy to draw this and check). However, if D is changed to D/2 we get $\mathcal{O}(D^2/h * h) = O(D^2)$, which is the same as the original without any heads, so we managed to get the same number of parameters as the one without heads as with heads when we altered D to D/2. Obviously this is more efficient because we end up using the exact same number of parameters, but we potentially have more expressive capability due to the number of heads we can use. Note that there is an extra cost incurred that is $\mathcal{O}(H)$, because we need to iterate through each head and perform self-attention, so that adds an extra multiplicative factor in addition to the time complexity of self-attention that is [$\mathcal{O}(N^2D)$](https://stackoverflow.com/questions/65703260/computational-complexity-of-self-attention-in-the-transformer-model), where N is the sequence length N and D is the dimensions of the data (features), making the time complexity for multiple heads $\mathcal{O}(N^2DH)$.


# 12.6
